In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math


train_path="../input/house-prices-advanced-regression-techniques/train.csv"
test_path="../input/house-prices-advanced-regression-techniques/test.csv"

train=pd.read_csv(train_path)
test=pd.read_csv(test_path)

In [3]:
#Copying Train And Test Datatset
c_train=train.copy()
c_test=test.copy()

In [4]:
train.shape

(1460, 81)

In [5]:
#To see the null values using a heatmap
#plt.figure(figsize = (16,7))
#sns.heatmap(train.isnull(),yticklabels=False,cbar=False)

* Lets Concat Train And Test Dataset

In [6]:
c_train["train"]=1
c_test["train"]=0
df=pd.concat([c_train,c_test],axis=0,sort=False)
#Vertically Concat

In [7]:
df.shape

(2919, 82)

83 = train/test features+train id +test id+SalePrice

In [8]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,train
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN,0
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN,0
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN,0
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN,0


* We Have Added Two New Features Train and Test To Show Which Values Are Present In Either Train Or Test

* Calculating the percentage of missing values of each feature

In [9]:
NAN=[(k,df[k].isnull().mean()*100) for k in df]
print(NAN)

[('Id', 0.0), ('MSSubClass', 0.0), ('MSZoning', 0.1370332305584104), ('LotFrontage', 16.649537512846866), ('LotArea', 0.0), ('Street', 0.0), ('Alley', 93.21685508735868), ('LotShape', 0.0), ('LandContour', 0.0), ('Utilities', 0.0685166152792052), ('LotConfig', 0.0), ('LandSlope', 0.0), ('Neighborhood', 0.0), ('Condition1', 0.0), ('Condition2', 0.0), ('BldgType', 0.0), ('HouseStyle', 0.0), ('OverallQual', 0.0), ('OverallCond', 0.0), ('YearBuilt', 0.0), ('YearRemodAdd', 0.0), ('RoofStyle', 0.0), ('RoofMatl', 0.0), ('Exterior1st', 0.0342583076396026), ('Exterior2nd', 0.0342583076396026), ('MasVnrType', 0.8221993833504625), ('MasVnrArea', 0.7879410757108599), ('ExterQual', 0.0), ('ExterCond', 0.0), ('Foundation', 0.0), ('BsmtQual', 2.774922918807811), ('BsmtCond', 2.8091812264474134), ('BsmtExposure', 2.8091812264474134), ('BsmtFinType1', 2.7064063035286057), ('BsmtFinSF1', 0.0342583076396026), ('BsmtFinType2', 2.7406646111682083), ('BsmtFinSF2', 0.0342583076396026), ('BsmtUnfSF', 0.034258

In [10]:
#Making a new dataframe
NAN=pd.DataFrame(NAN,columns=["column_name","percentage"])

* Displaying Features Which Has More Than 50% Null Values

In [11]:
NAN=NAN[NAN.percentage>50]
NAN.sort_values("percentage",ascending=False)

,column_name,percentage
72,PoolQC,99.657417
74,MiscFeature,96.402878
6,Alley,93.216855
73,Fence,80.438506


In [12]:
#Dropping All The Above Features
df=df.drop(["PoolQC","MiscFeature","Alley","Fence"],axis=1)

* Now we will select numerical and categorical features 

In [13]:
cat_columns = df.select_dtypes(include=['O'])
num_columns =df.select_dtypes(exclude=['O'])

# Dealing With Categorical Columns

In [14]:
null_cat_columns=cat_columns.isnull().sum()
null_cat_columns

MSZoning            4
Street              0
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         24
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           81
BsmtCond           82
BsmtExposure       82
BsmtFinType1       79
BsmtFinType2       80
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         1
Functional          2
FireplaceQu      1420
GarageType        157
GarageFinish      159
GarageQual        159
GarageCond        159
PavedDrive          0
SaleType            1
SaleCondition       0
dtype: int64

We will fill -- **BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, GarageType, GarageFinish, GarageQual, FireplaceQu, GarageCond** -- with "None" (Check The Above Output,We Place The Node In Features Where There Are Many NULL Values).

We will fill the rest of features with th most frequent value (using its own most frequent value).

In [15]:
columns_None = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
                'GarageType','GarageFinish','GarageQual','FireplaceQu','GarageCond']
cat_columns[columns_None]= cat_columns[columns_None].fillna('None')

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [16]:
mode_cols = ['MSZoning','Utilities','Exterior1st','Exterior2nd',
             'MasVnrType','Electrical','KitchenQual','Functional','SaleType']

#fill missing values for each column (using its own most frequent value)
cat_columns[mode_cols] = cat_columns[mode_cols].fillna(cat_columns.mode().iloc[0])

In [17]:
null_cat_columns=cat_columns.isnull().sum()
null_cat_columns

MSZoning         0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
KitchenQual      0
Functional       0
FireplaceQu      0
GarageType       0
GarageFinish     0
GarageQual       0
GarageCond       0
PavedDrive       0
SaleType         0
SaleCondition    0
dtype: int64

# Dealing With Num Vals

In [18]:
null_num_columns=num_columns.isnull().sum()
null_num_columns

Id                  0
MSSubClass          0
LotFrontage       486
LotArea             0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
MasVnrArea         23
BsmtFinSF1          1
BsmtFinSF2          1
BsmtUnfSF           1
TotalBsmtSF         1
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
GrLivArea           0
BsmtFullBath        2
BsmtHalfBath        2
FullBath            0
HalfBath            0
BedroomAbvGr        0
KitchenAbvGr        0
TotRmsAbvGrd        0
Fireplaces          0
GarageYrBlt       159
GarageCars          1
GarageArea          1
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
MiscVal             0
MoSold              0
YrSold              0
SalePrice        1459
train               0
dtype: int64

**By The Above Table We Can See That We Gotta Fill LotFrontage and GarageYrBlt**

In [19]:
print((num_columns["GarageYrBlt"]).median())
print(num_columns["LotFrontage"].median())

1979.0
68.0


*We Can Use Either Mean Or Median Depending Upon Us*

In [20]:
num_columns['GarageYrBlt'] = num_columns['GarageYrBlt'].fillna(1979)
num_columns['LotFrontage'] = num_columns['LotFrontage'].fillna(68)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Lets Fill The Other Null Num Features With 0

In [21]:
num_columns=num_columns.fillna(0)

In [22]:
num_columns.isnull().sum()

Id               0
MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
SalePrice        0
train            0
dtype: int64

# Lets Make Some New Features

In [23]:
num_columns["House_Age"]= num_columns['YrSold']- num_columns['YearBuilt']
num_columns["House_Age"].describe()

count    2919.000000
mean       36.479959
std        30.336182
min        -1.000000
25%         7.000000
50%        35.000000
75%        54.500000
max       136.000000
Name: House_Age, dtype: float64

* Lets Find If There Is A Negative Value In House_Age

In [24]:
neg_house=num_columns[num_columns["House_Age"]<0]
neg_house

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,train,House_Age
1089,2550,20,128.0,39290,10,5,2008,2009,1224.0,4010.0,...,0,0,0,0,17000,10,2007,0.0,0,-1


In [25]:
neg_house['YrSold']

1089    2007
Name: YrSold, dtype: int64

* Like we see here that the minimun is -1 ???
* It is strange to find that the house was sold in 2007 and built in 2008.
* So we decide to change the year of sold to 2009

In [26]:
num_columns.loc[num_columns["YrSold"]<num_columns["YearRemodAdd"],"YrSold"]=2009

num_columns["House_Age"]= num_columns['YrSold']- num_columns['YearBuilt']
num_columns["House_Age"].describe()

count    2919.000000
mean       36.482014
std        30.333755
min         0.000000
25%         7.000000
50%        35.000000
75%        54.500000
max       136.000000
Name: House_Age, dtype: float64

**According To The Feature Description**

* TotalBsmtBath : Sum of : 1/2 * BsmtFullBath + BsmtHalfBath

* TotalBath : Sum of : 1/2 * FullBath and HalfBath

* TotalSA : Sum of : 1stFlrSF and 2ndFlrSF and TotalBsmtSF

In [27]:
num_columns['TotalBsmtBath'] = num_columns['BsmtFullBath']*0.5 + num_columns['BsmtHalfBath']
num_columns['TotalBath'] = num_columns['FullBath']*0.5  + num_columns['HalfBath']
num_columns['TotalSA']=num_columns['TotalBsmtSF'] + num_columns['1stFlrSF'] + num_columns['2ndFlrSF']

In [28]:
num_columns.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,PoolArea,MiscVal,MoSold,YrSold,SalePrice,train,House_Age,TotalBsmtBath,TotalBath,TotalSA
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,...,0,0,2,2008,208500.0,1,5,0.5,2.0,2566.0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,...,0,0,5,2007,181500.0,1,31,1.0,1.0,2524.0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,...,0,0,9,2008,223500.0,1,7,0.5,2.0,2706.0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,...,0,0,2,2006,140000.0,1,91,0.5,0.5,2473.0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,...,0,0,12,2008,250000.0,1,8,0.5,2.0,3343.0


# Lets Encode Categorical Values

In [29]:
cat_columns.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Gd,Typ,None,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal


In [30]:
#Using One Hot Encoder
#cat_columns=pd.get_dummies(cat_columns,columns=cat_columns.columns)

* It simply creates additional features based on the number of unique values in the categorical feature. Every unique value in the category will be added as a feature.*

In [31]:
#Using LabelEncoder
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

for features in cat_columns.columns:
    cat_columns[features]=encoder.fit_transform(cat_columns[features])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


*Label Encoding is a popular encoding technique for handling categorical variables. In this technique, each label is assigned a unique integer based on alphabetical ordering.*

In [32]:
cat_columns.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,3,1,3,3,0,4,0,5,2,2,...,2,6,3,1,2,5,5,2,8,4
1,3,1,3,3,0,2,0,24,1,2,...,3,6,5,1,2,5,5,2,8,4
2,3,1,0,3,0,4,0,5,2,2,...,2,6,5,1,2,5,5,2,8,4
3,3,1,0,3,0,0,0,6,2,2,...,2,6,2,5,3,5,5,2,8,0
4,3,1,0,3,0,2,0,15,2,2,...,2,6,5,1,2,5,5,2,8,4


# Now Lets Concat The Num And Cat Features

In [33]:
df_final = pd.concat([cat_columns, num_columns], axis=1,sort=False)
df_final.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,PoolArea,MiscVal,MoSold,YrSold,SalePrice,train,House_Age,TotalBsmtBath,TotalBath,TotalSA
0,3,1,3,3,0,4,0,5,2,2,...,0,0,2,2008,208500.0,1,5,0.5,2.0,2566.0
1,3,1,3,3,0,2,0,24,1,2,...,0,0,5,2007,181500.0,1,31,1.0,1.0,2524.0
2,3,1,0,3,0,4,0,5,2,2,...,0,0,9,2008,223500.0,1,7,0.5,2.0,2706.0
3,3,1,0,3,0,0,0,6,2,2,...,0,0,2,2006,140000.0,1,91,0.5,0.5,2473.0
4,3,1,0,3,0,2,0,15,2,2,...,0,0,12,2008,250000.0,1,8,0.5,2.0,3343.0


# Initialising Train,Test Data

In [34]:
df_final = df_final.drop(['Id'],axis=1)

#We Had Initialised Train Data As 1 And Test Data As 0 In the Beggining
df_train = df_final[df_final['train'] == 1]
df_train = df_train.drop(['train'],axis=1)

#Vertically Splitting The Train And Test

df_test = df_final[df_final['train'] == 0]
df_test = df_test.drop(['SalePrice'],axis=1)
df_test = df_test.drop(['train'],axis=1)

In [35]:
df_train.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,House_Age,TotalBsmtBath,TotalBath,TotalSA
0,3,1,3,3,0,4,0,5,2,2,...,0,0,0,2,2008,208500.0,5,0.5,2.0,2566.0
1,3,1,3,3,0,2,0,24,1,2,...,0,0,0,5,2007,181500.0,31,1.0,1.0,2524.0
2,3,1,0,3,0,4,0,5,2,2,...,0,0,0,9,2008,223500.0,7,0.5,2.0,2706.0
3,3,1,0,3,0,0,0,6,2,2,...,0,0,0,2,2006,140000.0,91,0.5,0.5,2473.0
4,3,1,0,3,0,2,0,15,2,2,...,0,0,0,12,2008,250000.0,8,0.5,2.0,3343.0


In [36]:
df_test.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,House_Age,TotalBsmtBath,TotalBath,TotalSA
0,2,1,3,3,0,4,0,12,1,2,...,0,120,0,0,6,2010,49,0.0,0.5,1778.0
1,3,1,0,3,0,0,0,12,2,2,...,0,0,0,12500,6,2010,52,0.0,1.5,2658.0
2,3,1,0,3,0,4,0,8,2,2,...,0,0,0,0,3,2010,13,0.0,2.0,2557.0
3,3,1,0,3,0,4,0,8,2,2,...,0,0,0,0,6,2010,12,0.0,2.0,2530.0
4,3,1,0,1,0,4,0,22,2,2,...,0,144,0,0,1,2010,18,0.0,1.0,2560.0


*Now We Separate Train And Target Features*

In [37]:
target= df_train['SalePrice']
df_train = df_train.drop(['SalePrice'],axis=1)

# Modelling

In [38]:
x_train,x_test,y_train,y_test=train_test_split(df_train,target,test_size=0.33,random_state=0)

**XGB Documentation**: https://xgboost.readthedocs.io/en/latest/parameter.html

**LGBM Documentation**: https://lightgbm.readthedocs.io/en/latest/Parameters.html#core-parameters

In [39]:
xgb =XGBRegressor( booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=0.6, gamma=0,
                  importance_type='gain', learning_rate=0.01, max_delta_step=0,
                  max_depth=4, min_child_weight=1.5, n_estimators=2400,
                  n_jobs=1, nthread=None, objective='reg:linear',
                  reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
                  silent=None, subsample=0.8, verbosity=1)


lgbm = LGBMRegressor(objective='regression', 
                     num_leaves=4,
                     learning_rate=0.01, 
                     n_estimators=12000, 
                     max_bin=200, 
                     bagging_fraction=0.75,
                     bagging_freq=5, 
                     bagging_seed=7,
                     feature_fraction=0.4)

I Found This Article Very Helpful To Know About LGBM And XGBoost
https://www.analyticsvidhya.com/blog/2017/06/which-algorithm-takes-the-crown-light-gbm-vs-xgboost/                                                

In [40]:
#Fitting
xgb.fit(x_train, y_train)
lgbm.fit(x_train, y_train,eval_metric='rmse')

[07:18:23] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[07:18:29] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.4, learning_rate=0.01, max_bin=200,
              n_estimators=12000, num_leaves=4, objective='regression')

In [41]:
predict1 = xgb.predict(x_test)
predict = lgbm.predict(x_test)

In [42]:
print('Root Mean Square Error test(XGBOOST) = ' + str(math.sqrt(metrics.mean_squared_error(y_test, predict1))))
print('Root Mean Square Error test(LGBM) = ' + str(math.sqrt(metrics.mean_squared_error(y_test, predict))))

Root Mean Square Error test(XGBOOST) = 26810.40483625921
Root Mean Square Error test(LGBM) = 28084.508557740643


* Fitting With all the dataset 

In [43]:
xgb.fit(df_train, target)
lgbm.fit(df_train, target,eval_metric='rmse')

[07:18:34] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[07:18:42] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.4, learning_rate=0.01, max_bin=200,
              n_estimators=12000, num_leaves=4, objective='regression')

Root Mean Square Error (RMSE) is a standard way to measure the error of a model in predicting quantitative data.

Merging Both Predictions (45% of XGB and 55% of LGBM)

In [44]:
predict4 = lgbm.predict(df_test)
predict3 = xgb.predict(df_test)
predict_y = (predict3*0.45 + predict4*0.55)
print(predict_y)

[122960.98024405 167320.12980591 184224.60862451 ... 161106.42786976
 122446.88826078 216453.47350833]


In [45]:
submission = pd.DataFrame({
        "Id": test["Id"],
        "SalePrice": predict_y
    })
submission.to_csv('submission.csv', index=False)